## Project 10, task 5, Кривоногов Н.В. (hemerahu)

## 1. Создай соединение с базой данных с помощью библиотеки sqlite3.

In [1]:
import sqlite3

conn = sqlite3.connect('../data/checking-logs.sqlite')

conn

## 2. Используя только один запрос для каждой из групп, создай два датафрейма: test_results и control_results со столбцами time и avg_diff и только двумя строками.
- times должно иметь значения before и after.
- avg_diff содержит среднее значение дельты для всех пользователей за период времени до первого посещения ленты новостей каждым из них и после этого.
- Учитываются только те пользователи, для которых имеются наблюдения и before, и after.
## 3. Мы по-прежнему не используем лабораторное задание 'project1'.

In [2]:
import pandas as pd

test_results = pd.io.sql.read_sql('SELECT time, AVG(diff) AS avg_diff FROM \
                                  (SELECT uid, labname, AVG((strftime("%s", first_commit_ts) - deadlines) / 3600) AS diff, \
                                  CASE WHEN (strftime("%s", first_commit_ts) - strftime("%s", first_view_ts)) / 3600 < 0 \
                                  THEN "before" ELSE "after" END AS time FROM test AS t \
                                  LEFT JOIN deadlines AS d ON t.labname = d.labs WHERE labname IN \
                                  ("laba04", "laba04s", "laba05", "laba06", "laba06s") GROUP BY uid, time) \
                                  WHERE uid IN \
                                  (SELECT uid FROM (SELECT uid, \
                                  CASE WHEN (strftime("%s", first_commit_ts) - strftime("%s", first_view_ts)) / 3600 < 0 \
                                  THEN "before" ELSE "after" END AS time FROM test AS t WHERE labname IN \
                                  ("laba04", "laba04s", "laba05", "laba06", "laba06s") GROUP BY uid, time) \
                                  GROUP BY uid HAVING COUNT(uid) == 2) GROUP BY time;', conn)

test_results

,time,avg_diff
0,after,-99.523810
1,before,-66.047619


In [3]:
control_results = pd.io.sql.read_sql('SELECT time, AVG(diff) AS avg_diff FROM \
                                  (SELECT uid, labname, AVG((strftime("%s", first_commit_ts) - deadlines) / 3600) AS diff, \
                                  CASE WHEN (strftime("%s", first_commit_ts) - strftime("%s", first_view_ts)) / 3600 < 0 \
                                  THEN "before" ELSE "after" END AS time FROM control AS t \
                                  LEFT JOIN deadlines AS d ON t.labname = d.labs WHERE labname IN \
                                  ("laba04", "laba04s", "laba05", "laba06", "laba06s") GROUP BY uid, time) \
                                  WHERE uid IN \
                                  (SELECT uid FROM (SELECT uid, \
                                  CASE WHEN (strftime("%s", first_commit_ts) - strftime("%s", first_view_ts)) / 3600 < 0 \
                                  THEN "before" ELSE "after" END AS time FROM control AS t WHERE labname IN \
                                  ("laba04", "laba04s", "laba05", "laba06", "laba06s") GROUP BY uid, time) \
                                  GROUP BY uid HAVING COUNT(uid) == 2) GROUP BY time;', conn)

control_results

,time,avg_diff
0,after,-99.322222
1,before,-98.033333


## 4. Закрой соединение.

In [4]:
conn.close()

## 5. Дайте ответ: оказалось ли предположение верным и влияет ли наличие страницы с новостной лентой на поведение учащихся?

Предположение оказалось верным. Ведь в тестовой выборке средняя дельта показывает, что посещение ленты новостей оказало на поведение учащихся. В итоге они приступили к работе над лабораторным заданием раньше. 